In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import optuna

# 导入训练集和测试集
train_data = pd.read_excel(r'C:\Users\k\Desktop\硕士论文\ORP\ORP4\XGBoost\ORP_function_train.xlsx')
test_data = pd.read_excel(r'C:\Users\k\Desktop\硕士论文\ORP\ORP4\XGBoost\ORP_function_test.xlsx')

# 分离特征和目标变量
X_train, y_train = train_data.drop(['E','File'], axis=1), train_data['E']
X_test, y_test = test_data.drop(['E','File'], axis=1), test_data['E']

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'gamma': trial.suggest_float('gamma', 0, 0.6),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0)
    }

    model = XGBRegressor(**param)
    
    # 使用k折交叉验证
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold)
        y_val_pred = model.predict(X_val_fold)
        rmse_fold = np.sqrt(mean_squared_error(y_val_fold, y_val_pred))
        rmse_scores.append(rmse_fold)
    
    # 计算交叉验证的平均RMSE
    avg_rmse = np.mean(rmse_scores)
    
    return avg_rmse

# 创建Optuna试验
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

# 获取最佳超参数
best_params = study.best_params
print("最佳超参数组合：", best_params)

# 使用最佳超参数训练模型  
best_xgb_reg = XGBRegressor(**best_params)
best_xgb_reg.fit(X_train, y_train)  

# 预测训练集和测试集数据  
y_train_pred = best_xgb_reg.predict(X_train)  
y_test_pred = best_xgb_reg.predict(X_test)  

# 计算训练集和测试集的均方根误差 (RMSE) 和决定系数 (R2)  
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))  
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))  

train_r2 = r2_score(y_train, y_train_pred)  
test_r2 = r2_score(y_test, y_test_pred)  

# 输出 RMSE 和 R2  
print("训练集的均方根误差 (RMSE)：", train_rmse)  
print("测试集的均方根误差 (RMSE)：", test_rmse)  
print("训练集的决定系数 (R2)：", train_r2)  
print("测试集的决定系数 (R2)：", test_r2)


[I 2024-02-23 13:22:30,293] A new study created in memory with name: no-name-14038941-f89a-492a-a686-0501ec98bdbb
[I 2024-02-23 13:22:30,658] Trial 0 finished with value: 0.4519368704517827 and parameters: {'learning_rate': 0.6942182695576309, 'n_estimators': 354, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.14993827857446987, 'subsample': 0.9527089051620541, 'colsample_bytree': 0.913495060424622}. Best is trial 0 with value: 0.4519368704517827.
[I 2024-02-23 13:22:30,943] Trial 1 finished with value: 0.4211808307871093 and parameters: {'learning_rate': 0.0716074809895613, 'n_estimators': 218, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.024927691754009638, 'subsample': 0.6510436243419037, 'colsample_bytree': 0.8648805079485005}. Best is trial 1 with value: 0.4211808307871093.
[I 2024-02-23 13:22:31,411] Trial 2 finished with value: 0.5143899606272153 and parameters: {'learning_rate': 0.8932081757158494, 'n_estimators': 434, 'max_depth': 5, 'min_child_weight': 2, 'gamma': 0.0

[I 2024-02-23 13:22:39,759] Trial 24 finished with value: 0.43953862136100774 and parameters: {'learning_rate': 0.3331266588838966, 'n_estimators': 450, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.07389015901418182, 'subsample': 0.6314116559230839, 'colsample_bytree': 0.7315778225287325}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:22:40,183] Trial 25 finished with value: 0.42936174198659016 and parameters: {'learning_rate': 0.10953295072563723, 'n_estimators': 346, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.05947163914780375, 'subsample': 0.770737192861877, 'colsample_bytree': 0.6497037557062243}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:22:40,656] Trial 26 finished with value: 0.429122920836215 and parameters: {'learning_rate': 0.21563907458485, 'n_estimators': 416, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.05079600856577307, 'subsample': 0.748829025284361, 'colsample_bytree': 0.7480626195639728}. Best is trial 23 wi

[I 2024-02-23 13:22:51,507] Trial 48 finished with value: 0.4518731868529053 and parameters: {'learning_rate': 0.3072021145438476, 'n_estimators': 355, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.10777973203627568, 'subsample': 0.9911173211415834, 'colsample_bytree': 0.6866400445941263}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:22:51,912] Trial 49 finished with value: 0.41946198739098 and parameters: {'learning_rate': 0.18173852217780506, 'n_estimators': 367, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.07721612051709922, 'subsample': 0.7095877296396412, 'colsample_bytree': 0.650310693167401}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:22:52,239] Trial 50 finished with value: 0.41922059135733347 and parameters: {'learning_rate': 0.10680103929171392, 'n_estimators': 280, 'max_depth': 2, 'min_child_weight': 5, 'gamma': 0.17576508769117, 'subsample': 0.6670102989295228, 'colsample_bytree': 0.7083105687567317}. Best is trial 23 wit

[I 2024-02-23 13:23:03,233] Trial 72 finished with value: 0.41082185725861364 and parameters: {'learning_rate': 0.05588773235421343, 'n_estimators': 474, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.12093710873011798, 'subsample': 0.6367104915793833, 'colsample_bytree': 0.670109086891904}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:23:03,901] Trial 73 finished with value: 0.41699730336515656 and parameters: {'learning_rate': 0.012248604208432625, 'n_estimators': 500, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.07104292108582158, 'subsample': 0.6115481642177072, 'colsample_bytree': 0.6154032109778222}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:23:04,390] Trial 74 finished with value: 0.4303215045179959 and parameters: {'learning_rate': 0.16716598629729068, 'n_estimators': 449, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.09294117634959723, 'subsample': 0.6636176845018388, 'colsample_bytree': 0.6601808812543062}. Best is tria

[I 2024-02-23 13:23:15,185] Trial 96 finished with value: 0.4169962730178611 and parameters: {'learning_rate': 0.058386696414214546, 'n_estimators': 466, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.1295706068903958, 'subsample': 0.6504320345580691, 'colsample_bytree': 0.8081092521981331}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:23:15,699] Trial 97 finished with value: 0.4746028718719314 and parameters: {'learning_rate': 0.5952861521017553, 'n_estimators': 490, 'max_depth': 2, 'min_child_weight': 2, 'gamma': 0.0996328037060123, 'subsample': 0.6338373512160626, 'colsample_bytree': 0.8157631269609994}. Best is trial 23 with value: 0.4040517724525937.
[I 2024-02-23 13:23:16,167] Trial 98 finished with value: 0.4239633442644962 and parameters: {'learning_rate': 0.24410383905864474, 'n_estimators': 447, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.11368036672459951, 'subsample': 0.6235870004880508, 'colsample_bytree': 0.6481550112928278}. Best is trial 23

...

[I 2024-02-23 13:29:56,218] Trial 912 finished with value: 0.41311829484268464 and parameters: {'learning_rate': 0.019188283287051244, 'n_estimators': 469, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.11231563338280906, 'subsample': 0.6256957766405431, 'colsample_bytree': 0.6633695451943801}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:29:56,497] Trial 913 finished with value: 0.41446245700396245 and parameters: {'learning_rate': 0.09693145698566902, 'n_estimators': 122, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.13882752272642254, 'subsample': 0.6183686199410839, 'colsample_bytree': 0.8636408009351797}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:29:57,121] Trial 914 finished with value: 0.4090725909271171 and parameters: {'learning_rate': 0.07667392256572457, 'n_estimators': 493, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.08968712758729752, 'subsample': 0.6432234948443482, 'colsample_bytree': 0.6751394809343445}. Best

[I 2024-02-23 13:30:11,219] Trial 936 finished with value: 0.4158319727597714 and parameters: {'learning_rate': 0.11301031535160691, 'n_estimators': 479, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.07981676264930918, 'subsample': 0.637230290761255, 'colsample_bytree': 0.6881473212281604}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:30:11,996] Trial 937 finished with value: 0.4169928234447056 and parameters: {'learning_rate': 0.019723118745862328, 'n_estimators': 487, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.1317447009073295, 'subsample': 0.6080115907952394, 'colsample_bytree': 0.6764875404627028}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:30:12,654] Trial 938 finished with value: 0.4100964174764237 and parameters: {'learning_rate': 0.07152651387384032, 'n_estimators': 473, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.10262770972680055, 'subsample': 0.6070762694300627, 'colsample_bytree': 0.6611889975009136}. Best is 

[I 2024-02-23 13:30:27,510] Trial 960 finished with value: 0.40602692335567736 and parameters: {'learning_rate': 0.07426743454442797, 'n_estimators': 464, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.1300504288657179, 'subsample': 0.6111664095002091, 'colsample_bytree': 0.6469130681295849}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:30:28,082] Trial 961 finished with value: 0.4164285012981478 and parameters: {'learning_rate': 0.07881554538653657, 'n_estimators': 477, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.1138958766454435, 'subsample': 0.6287402662740862, 'colsample_bytree': 0.8112633544849905}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:30:28,654] Trial 962 finished with value: 0.41509437420430667 and parameters: {'learning_rate': 0.10040614342195003, 'n_estimators': 483, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.1404563347611634, 'subsample': 0.620785641696237, 'colsample_bytree': 0.6542228680466958}. Best is t

[I 2024-02-23 13:30:42,158] Trial 984 finished with value: 0.40987919996549504 and parameters: {'learning_rate': 0.08450745755946243, 'n_estimators': 403, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.0821738728655765, 'subsample': 0.6211705150758472, 'colsample_bytree': 0.6956225926374121}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:30:42,815] Trial 985 finished with value: 0.40758208583468647 and parameters: {'learning_rate': 0.05622519972744934, 'n_estimators': 493, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.10327455426142448, 'subsample': 0.6313403398470495, 'colsample_bytree': 0.684727957719397}. Best is trial 394 with value: 0.39737679572967455.
[I 2024-02-23 13:30:43,460] Trial 986 finished with value: 0.4060632849756143 and parameters: {'learning_rate': 0.0490872834563148, 'n_estimators': 488, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.07242185996188147, 'subsample': 0.6198403096638746, 'colsample_bytree': 0.6699187939236386}. Best is 

最佳超参数组合： {'learning_rate': 0.056870729222653196, 'n_estimators': 489, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.10590750214578348, 'subsample': 0.6221487283126023, 'colsample_bytree': 0.6580031845191148}
训练集的均方根误差 (RMSE)： 0.22889587270685205
测试集的均方根误差 (RMSE)： 0.4422602420890535
训练集的决定系数 (R2)： 0.9201661329434412
测试集的决定系数 (R2)： 0.7112128160166157
